In [34]:
import pandas as pd
import itertools
from collections import Counter
import json
from dotenv import load_dotenv
import os
import openai
import os
import matplotlib.pyplot as plt
import requests
import azure
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
import evaluate
import urllib.request
import ssl
from scipy import stats

#### Load configuration file

In [17]:
import os
import yaml
from datetime import datetime
snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open('config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_DATA_NAME']    
DATA_DIR = d['config']['DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']
IS_DEBUG = d['config']['IS_DEBUG']
USE_LOWPRIORITY_VM = d['config']['USE_LOWPRIORITY_VM']


rest_endpoint = d['evaluation']['rest_endpoint']
evaluation_deployment_name = d['evaluation']['deployment_name']

### Configure workspace

In [18]:
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import command
from azure.ai.ml.entities import Data, Environment, BuildContext
from azure.ai.ml.entities import Model
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

Found the config file in: /config.json


### Format csv to json

In [5]:
import pandas as pd
df = pd.read_csv("analysis_and_tools_only.csv")

In [19]:
#preprocess data

input_string = []

# Iterate through the rows of the dataframe
for index, row in df.iterrows():
    # 'content' is the column with the user questions
    user_content = row['content']
    # 'answer content' is the column with the ground truth answers
    assistant_content = row['answer_content']

    # Append only user content to the input_data
    input_string.append({"role": "user", "content": user_content})

# Define the parameters for the model
params = {
    "temperature": 0.1,
    "max_new_tokens": 4096, #output
    "do_sample": True,
    "return_full_text": False
}

# Combine input data and parameters into the final test data structure
test_data = {
    "input_data": input_string,
    "parameters": params
}

# Save the JSON output to a file
with open('test_data.json', 'w') as json_file:
    json.dump(test_data, json_file, indent=4)

print("JSON file created successfully.")

JSON file created successfully.


In [20]:
import requests
import json
from azure.identity import DefaultAzureCredential

# Replace with your actual OpenAI endpoint URL and deployment details
endpoint_url = "<YOUR_OPENAI_ENDPOINT_URL>"
deployment_name = "<YOUR_CHAT_MODEL_DEPLOYMENT_NAME>"  # Make sure this is a chat model like 'gpt-4-turbo'

# Authenticate using AAD (Azure Active Directory)
credential = DefaultAzureCredential()
token = credential.get_token("https://cognitiveservices.azure.com/.default").token

# Define the chat input in the appropriate format for chat completion
input_data = {
    "messages": [
        {"role": "user", "content": "What is samtools."}
    ],
    "max_tokens": 4096,
    "temperature": 0.1
    }

# Set up the headers with AAD token
headers = {
    "Authorization": f"Bearer {token}",
    "Content-Type": "application/json"
}

# Send a request to the chat completion API
response = requests.post(
    url=f"https://olmelnic-test.openai.azure.com/openai/deployments/gpt-4o-2024-05-13/chat/completions?api-version=2023-03-15-preview",
    headers=headers,
    json=input_data
)

# Print the response
if response.status_code == 200:
    result = response.json()
    print(result["choices"][0]["message"]["content"])
else:
    print(f"Failed with status code: {response.status_code}, {response.text}")


Failed with status code: 429, {"error":{"code":"429","message": "Rate limit is exceeded. Try again in 8 seconds."}}


### Evaluate model 

#### Evaulation real data

In [8]:
# Define the path for saving the real request file
test_src_dir = "./gpt4-inference-test"
os.makedirs(test_src_dir, exist_ok=True)
real_data_path = os.path.join(test_src_dir, "test_data.json")

# Save the real request data to a JSON file
with open(real_data_path, "w") as f:
    json.dump(test_data, f, indent=4)

In [9]:
# Define paths and parameters
timeout = 1000
test_src_dir = "./gpt4-inference-test"
response_src_dir = "./gpt4-inference-responses"
batch_size = 1

# Ensure directories exist
os.makedirs(test_src_dir, exist_ok=True)
os.makedirs(response_src_dir, exist_ok=True)

def batch_data(data, batch_size):
    """Split data into batches of given size."""
    return [data[i:i + batch_size] for i in range(0, len(data), batch_size)]


# Create batches
batches = batch_data(test_data["input_data"], batch_size)

In [21]:
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.


In [23]:
import os
import json
import urllib.request
import time
from azure.identity import DefaultAzureCredential

def submit_tasks(query, id, max_retries=5):
    data = {
    "messages": query,
    "max_tokens": 4096,
    "temperature": 0.1
    }


    batch_file_path = os.path.join(test_src_dir, f"batch_{id}.json")
    # Save the batch request data to a JSON file
    with open(batch_file_path, "w") as f:
        json.dump(data, f, indent=4)

    body = str.encode(json.dumps(data))
    #print(f"Batch request data saved to: {batch_file_path}")

    url = 'https://olmelnic-test.openai.azure.com/openai/deployments/gpt-4o-2024-05-13/chat/completions?api-version=2023-06-01-preview'
    
    # Authenticate using AAD (Azure Active Directory)
    credential = DefaultAzureCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default").token

    # Set up the headers with AAD token
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    retries = 0
    while retries < max_retries:
        response = requests.post(
            url=f"https://olmelnic-test.openai.azure.com/openai/deployments/gpt-4o-2024-05-13/chat/completions?api-version=2023-03-15-preview",
            headers=headers,
            json=data
        )
        # Check if the response is rate limited (status code 429)
        if response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", 32))  # Default to 32 seconds if not provided
            print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
            retries += 1
        else:
            # If successful or another error occurred, break out of the retry loop
            break

    # Check if the request was successful after retrying
    if response.status_code == 200:
        result = response.json()
        output = result["choices"][0]["message"]["content"]
    else:
        print(f"Request failed after {retries} retries. Status code: {response.status_code}, Response: {response.text}")
 
    response_file_path = os.path.join(response_src_dir, f"response_{id}.json")
    
    # Save both input and response data
    with open(response_file_path, "w") as f:
        json.dump({
            "input_data": query,  # Use query here
            "response_data": output  # Ensure result is JSON serializable
        }, f, indent=4)
    
    print(f"Result for batch {id} saved to: {response_file_path}")



In [14]:
## loop through batches
for i, batch in enumerate(batches):
    submit_tasks(batch,i)

Batch request data saved to: ./gpt4-inference-test/batch_0.json
Result for batch 0 saved to: ./gpt4-inference-responses/response_0.json
Batch request data saved to: ./gpt4-inference-test/batch_1.json
Result for batch 1 saved to: ./gpt4-inference-responses/response_1.json
Batch request data saved to: ./gpt4-inference-test/batch_2.json
Rate limit exceeded. Retrying in 47 seconds...
Result for batch 2 saved to: ./gpt4-inference-responses/response_2.json
Batch request data saved to: ./gpt4-inference-test/batch_3.json
Rate limit exceeded. Retrying in 1 seconds...
Result for batch 3 saved to: ./gpt4-inference-responses/response_3.json
Batch request data saved to: ./gpt4-inference-test/batch_4.json
Rate limit exceeded. Retrying in 45 seconds...
Result for batch 4 saved to: ./gpt4-inference-responses/response_4.json
Batch request data saved to: ./gpt4-inference-test/batch_5.json
Rate limit exceeded. Retrying in 2 seconds...
Result for batch 5 saved to: ./gpt4-inference-responses/response_5.jso

In [12]:
import glob
import pandas as pd

file_pattern = "./gpt4-inference-responses/*.json"

questions = []
answers = []

for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
    with open(file_name, 'r') as file:
        data = json.load(file)
    

    if "input_data" in data: #retrieves list of dictionaries
        for item in data["input_data"]: #iterates over each dictionary in the list
            if "content" in item:
                questions.append(item["content"])

    if "response_data" in data:
        response = data["response_data"]
        answers.append(response)

final_df = pd.DataFrame({"Question": questions, "Answer": answers})

# Save DataFrame to a CSV file (optional)
final_df.to_csv('combined_questions_answers.csv', index=False)

final_df.head()

,Question,Answer
0,Hi Everyone. I was trying to add help section ...,The issue you're encountering is due to the pl...
1,I am currently using BWA-MEM to map metagenomi...,"Yes, you can filter out the unmapped reads to ..."
2,Hi all:I recently got quite confused with two ...,"Hi Vanilla,\n\nYour confusion is understandabl..."
3,"Hello, I am pretty new to bioinformatics and t...",Welcome to the world of bioinformatics! It sou...
4,Samtools can be used to select reads above cer...,To select reads with a mapping quality below a...


#### ROUGE

########note: rouge captures (number of n-grams in prediction summary (fine-tuned phi-3 model) that match the reference summary (ground-truth)) / number of n-grams in reference summary

In [13]:
predictions = final_df["Answer"].to_list()
references = df["answer_content"].to_list()

rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge


results = rouge.compute(predictions=predictions,
                         references=references,
                        use_aggregator=True)

print("ROUGE-1:", round(results["rouge1"], 3))
print("ROUGE-2:", round(results["rouge2"], 3))
print("ROUGE-L:", round(results["rougeL"], 3))
print("ROUGE-Lsum:", round(results["rougeLsum"], 3))

ROUGE-1: 0.122
ROUGE-2: 0.014
ROUGE-L: 0.072
ROUGE-Lsum: 0.091


### Loop 100 times for generating confidence intervals

In [25]:
import glob
import pandas as pd

rouge_df = pd.DataFrame(columns=["ROUGE-1", "ROUGE-2", "ROUGE-L", "ROUGE-Lsum"])

for j in range(100):

    print(f"Iteration {j}")

    ## loop through batches
    for i, batch in enumerate(batches):
        submit_tasks(batch,i)

    file_pattern = "./gpt4-inference-responses/*.json"

    questions = []
    answers = []

    for file_name in glob.glob(file_pattern): #glob lists all the text files in the current working dir.
        with open(file_name, 'r') as file:
            data = json.load(file)
        

        if "input_data" in data: #retrieves list of dictionaries
            for item in data["input_data"]: #iterates over each dictionary in the list
                if "content" in item:
                    questions.append(item["content"])

        if "response_data" in data:
            response = data["response_data"]
            answers.append(response)

    final_df = pd.DataFrame({"Question": questions, "Answer": answers})

    # Save DataFrame to a CSV file (optional)
    final_df.to_csv('combined_questions_answers.csv', index=False)

    final_df.head()

    predictions = final_df["Answer"].to_list()
    references = df["answer_content"].to_list()

    rouge = evaluate.load('rouge') #https://huggingface.co/spaces/evaluate-metric/rouge


    results = rouge.compute(predictions=predictions,
                            references=references,
                            use_aggregator=True)
    rouge_df.loc[j] = [
        round(results["rouge1"], 3),
        round(results["rouge2"], 3),
        round(results["rougeL"], 3),
        round(results["rougeLsum"], 3)
    ]

Iteration 0
Rate limit exceeded. Retrying in 12 seconds...
Result for batch 0 saved to: ./gpt4-inference-responses/response_0.json
Rate limit exceeded. Retrying in 23 seconds...
Rate limit exceeded. Retrying in 30 seconds...
Rate limit exceeded. Retrying in 30 seconds...


KeyboardInterrupt: 

In [32]:
# Calculate Confidence Interval
def confidence_interval(data, confidence=0.95):
    mean = np.mean(data)
    sem = stats.sem(data)  # Standard Error of Mean
    interval = sem * stats.t.ppf((1 + confidence) / 2., len(data) - 1)
    return mean, interval

In [ ]:
rouge_df.to_csv('./rouge_results.csv', index=False)


In [28]:
rouge_df = pd.read_csv('./rouge_results.csv')

In [35]:
import numpy as np
import glob
import pandas as pd

rouge1_mean, rouge1_ci = confidence_interval(rouge_df['ROUGE-1'])
rouge2_mean, rouge2_ci = confidence_interval(rouge_df['ROUGE-2'])
rougeL_mean, rougeL_ci = confidence_interval(rouge_df['ROUGE-L'])
rougeLsum_mean, rougeLsum_ci = confidence_interval(rouge_df['ROUGE-Lsum'])

print(f"ROUGE-1: {rouge1_mean:.3f} ± {rouge1_ci:.3f}")
print(f"ROUGE-2: {rouge2_mean:.3f} ± {rouge2_ci:.3f}")
print(f"ROUGE-L: {rougeL_mean:.3f} ± {rougeL_ci:.3f}")
print(f"ROUGE-Lsum: {rougeLsum_mean:.3f} ± {rougeLsum_ci:.3f}")

ROUGE-1: 0.122 ± 0.000
ROUGE-2: 0.014 ± 0.000
ROUGE-L: 0.070 ± 0.000
ROUGE-Lsum: 0.089 ± 0.000
